# Experimenting w tkinter

## Simple canvas

The code below outputs a canvas in a separate window.


In [45]:
# from tkinter import * #originally wildcard import
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

constants
def savePosn(event):
    global lastx, lasty
    lastx, lasty = event.x, event.y

def addLine(event):
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()

# root.columnconfigure(0, weight=1)
# root.rowconfigure(0, weight=1)

# canvas = Canvas(root)
# canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S ))
# canvas.bind("<Button-1>", savePosn)
# canvas.bind("<B1-Motion>", addLine)

root.mainloop()

## Output 1:

The root object with the mainloop() method will open a window where drawing takes place.

<img src="./figures/tkinter_canvas_1.png" alt="Kitten" title="A cute kitten" width="400" height="100" /> 


## Another canvas

This canvas encapsulates some of the code above so global variables are not used.

In [ ]:
# from tkinter import *
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

class Sketchpad(Canvas):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.bind("<Button-1>", self.save_posn)
        self.bind("<B1-Motion>", self.add_line)
        
    def save_posn(self, event):
        self.lastx, self.lasty = event.x, event.y

    def add_line(self, event):
        self.create_line((self.lastx, self.lasty, event.x, event.y))
        self.save_posn(event)

root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

sketch = Sketchpad(root)
sketch.grid(column=0, row=0, sticky=(con.N, con.W, con.E, con.S))

root.mainloop()

## Output 2:

<img src="./figures/tkinter_canvas_2.png" alt="" title="" width="400" height="100" /> 